In [13]:

import torch
import torch.nn.functional as F
import argparse
import numpy as np
import pandas as pd
from IPython.display import display
import warnings
warnings.filterwarnings("ignore")

from data_utils import build_tokenizer, build_embedding_matrix, Tokenizer4Bert, pad_and_truncate
from models import IAN


from dependency_graph import dependency_adj_matrix

from transformers import BertModel

model_output = None

class CharVal(object):
    def __init__(self, char, val):
        self.char = char
        self.val = val

    def __str__(self):
        return self.char

def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % rgb
def color_charvals(s):
    r = 255-int(s.val*255)
    color = rgb_to_hex((255, r, r))
    return 'background-color: %s' % color


# before the prediction i supposed you tokenized text
# you need to match each char and attention


def model_hook(module, input_, output):
    global model_output
    model_output = output

class Inferer:
    """A simple inference example"""
    def __init__(self, opt):
        self.opt = opt
        if 'bert' in opt.model_name:
            self.tokenizer = Tokenizer4Bert(opt.max_seq_len, opt.pretrained_bert_name)
            bert = BertModel.from_pretrained(opt.pretrained_bert_name)
            self.model = opt.model_class(bert, opt).to(opt.device)
        else:
            self.tokenizer = build_tokenizer(
                fnames=[opt.dataset_file['train'], opt.dataset_file['test']],
                max_seq_len=opt.max_seq_len,
                dat_fname='{0}_tokenizer.dat'.format(opt.dataset))
            embedding_matrix = build_embedding_matrix(
                word2idx=self.tokenizer.word2idx,
                embed_dim=opt.embed_dim,
                dat_fname='{0}_{1}_embedding_matrix.dat'.format(str(opt.embed_dim), opt.dataset))
            self.model = opt.model_class(embedding_matrix, opt)
        print('loading model {0} ...'.format(opt.model_name))
        self.model.load_state_dict(torch.load(opt.state_dict_path))
        self.model = self.model.to(opt.device)
        # switch model to evaluation mode
        self.model.eval()
        print(self.model)
        torch.autograd.set_grad_enabled(False)

    def evaluate(self, text, aspect):
        aspect = aspect.lower().strip()
        text_left, _, text_right = [s.strip() for s in text.lower().partition(aspect)]

        text_indices = self.tokenizer.text_to_sequence(text_left + " " + aspect + " " + text_right)
        context_indices = self.tokenizer.text_to_sequence(text_left + " " + text_right)
        left_indices = self.tokenizer.text_to_sequence(text_left)
        left_with_aspect_indices = self.tokenizer.text_to_sequence(text_left + " " + aspect)
        right_indices = self.tokenizer.text_to_sequence(text_right, reverse=True)
        right_with_aspect_indices = self.tokenizer.text_to_sequence(aspect + " " + text_right, reverse=True)
        aspect_indices = self.tokenizer.text_to_sequence(aspect)
        left_len = np.sum(left_indices != 0)
        aspect_len = np.sum(aspect_indices != 0)
        aspect_boundary = np.asarray([left_len, left_len + aspect_len - 1], dtype=np.int64)

        text_len = np.sum(text_indices != 0)
        concat_bert_indices = self.tokenizer.text_to_sequence('[CLS] ' + text_left + " " + aspect + " " + text_right + ' [SEP] ' + aspect + " [SEP]")
        concat_segments_indices = [0] * (text_len + 2) + [1] * (aspect_len + 1)
        concat_segments_indices = pad_and_truncate(concat_segments_indices, self.tokenizer.max_seq_len)

        text_bert_indices = self.tokenizer.text_to_sequence("[CLS] " + text_left + " " + aspect + " " + text_right + " [SEP]")
        aspect_bert_indices = self.tokenizer.text_to_sequence("[CLS] " + aspect + " [SEP]")

        dependency_graph = dependency_adj_matrix(text)

        data = {
            'concat_bert_indices': concat_bert_indices,
            'concat_segments_indices': concat_segments_indices,
            'text_bert_indices': text_bert_indices,
            'aspect_bert_indices': aspect_bert_indices,
            'text_indices': text_indices,
            'context_indices': context_indices,
            'left_indices': left_indices,
            'left_with_aspect_indices': left_with_aspect_indices,
            'right_indices': right_indices,
            'right_with_aspect_indices': right_with_aspect_indices,
            'aspect_indices': aspect_indices,
            'aspect_boundary': aspect_boundary,
            'dependency_graph': dependency_graph,
        }

        t_inputs = [torch.tensor([data[col]], device=self.opt.device) for col in self.opt.inputs_cols]
        self.model.attention_context.register_forward_hook(model_hook)
        t_outputs = self.model(t_inputs)
        t_probs = F.softmax(t_outputs, dim=-1).cpu().numpy()

        return t_probs


if __name__ == '__main__':
    dataset_files = {
        'restaurant': {
            'train': './datasets/semeval14/Restaurants_Train.xml.seg',
            'test': './datasets/semeval14/Restaurants_Test_Gold.xml.seg'
        },
        'laptop': {
            'train': './datasets/semeval14/Laptops_Train.xml.seg',
            'test': './datasets/semeval14/Laptops_Test_Gold.xml.seg'
        }
    }
    class Option(object): pass
    opt = Option()
    opt.model_name = 'ian'
    opt.model_class = IAN
    opt.dataset = 'laptop'
    opt.dataset_file = dataset_files["laptop"]
    opt.inputs_cols = ['text_indices', 'aspect_indices']
    # set your trained models here
    opt.state_dict_path = 'state_dict/ian_laptop_val_acc_0.6426'
    opt.embed_dim = 300
    opt.hidden_dim = 300
    opt.max_seq_len = 85
    opt.bert_dim = 768
    opt.pretrained_bert_name = 'bert-base-uncased'
    opt.polarities_dim = 3
    opt.hops = 3
    opt.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    opt.local_context_focus = 'cdm'
    opt.SRD = 3

    inf = Inferer(opt)
    sentence_r = ['I did not like the food, but service was good.',
                "The food was pathetic",
                "They have very good icecream"]
    sentence_l = ["The laptop runs very fast",
                "I am fed up up with os"]
    aspect_r = ["food","food","icecream"]
    aspect_l = ["laptop", "os"]
    for s, a in zip(sentence_l, aspect_l):
        t_probs = inf.evaluate(s, a)
        print(t_probs.argmax(axis=-1) - 1)
        print(model_output[1].squeeze().cpu())
        char_vals = [CharVal(c, v) for c, v in zip(s.split(), model_output[1].squeeze().cpu())]
        char_df = pd.DataFrame(char_vals).transpose()
        # apply coloring values
        char_df = char_df.style.applymap(color_charvals)
        display(char_df)

loading tokenizer: laptop_tokenizer.dat
loading embedding_matrix: 300_laptop_embedding_matrix.dat
loading model ian ...
IAN(
  (embed): Embedding(3600, 300)
  (lstm_context): DynamicLSTM(
    (RNN): LSTM(300, 300, batch_first=True)
  )
  (lstm_aspect): DynamicLSTM(
    (RNN): LSTM(300, 300, batch_first=True)
  )
  (attention_aspect): Attention(
    (w_k): Linear(in_features=300, out_features=300, bias=True)
    (w_q): Linear(in_features=300, out_features=300, bias=True)
    (proj): Linear(in_features=300, out_features=300, bias=True)
    (dropout): Dropout(p=0, inplace=False)
  )
  (attention_context): Attention(
    (w_k): Linear(in_features=300, out_features=300, bias=True)
    (w_q): Linear(in_features=300, out_features=300, bias=True)
    (proj): Linear(in_features=300, out_features=300, bias=True)
    (dropout): Dropout(p=0, inplace=False)
  )
  (dense): Linear(in_features=600, out_features=3, bias=True)
)
[1]
tensor([0.0236, 0.0394, 0.0587, 0.2149, 0.6634])
[0]
tensor([0.0408, 0.

,0,1,2,3,4
0,The,laptop,runs,very,fast


,0,1,2,3,4,5,6
0,I,am,fed,up,up,with,os
